<a href="https://colab.research.google.com/github/Witcape/AI_Practice/blob/main/PSO_With_Llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install accelerate
# import accelerate

In [2]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 57.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.0 MB/s eta 0:00:00
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-2b79p27h/overlay/local/bin
    changing mode of /tmp/pip-build-env-2b79p27h/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-2b79p27h/overlay/local/bin/wheel to 755
    changing mode of /tmp/pip

In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [5]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:
lcpp_llm.params.n_gpu_layers

32

In [19]:
prompt = "generate 50 numbers between -10 to 10 by using Chaotic Number Generator, only 2 decimal places"
prompt_template = f'''SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

In [20]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [21]:
print(response)

{'id': 'cmpl-2ad7c8ed-0fd1-4619-b546-aa38d0887f2d', 'object': 'text_completion', 'created': 1718388377, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.\n\nUSER: generate 50 numbers between -10 to 10 by using Chaotic Number Generator, only 2 decimal places\n\nASSISTANT:\n\nI'm happy to help! Here are 50 chaotic numbers between -10 and 10 with two decimal places:\n\n1. -9.48\n2. -7.36\n3. -5.23\n4. -3.11\n5. -1.02\n6. 1.98\n7. 3.94\n8. 5.91\n9. 7.87\n10. 9.83\n11. -0.12\n12. 0.21\n13. 0.46\n14. 0.72\n15. 0.98\n16. 1.24\n17. 1.49\n18. 1.75\n19. 2.02\n20. -0.31\n21. -0.07\n22. 0.14\n23. 0.28\n24. 0.42\n25. 0.56\n26. 0.69\n", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 60, 'completion_tokens': 256, 'total_tokens': 316}}


In [24]:
import re

text_response = response['choices'][0]['text']

# Use a regular expression to find all numbers in the response
numbers = re.findall(r'-?\d+\.\d+', text_response)

# Convert the numbers to floats
numbers = [float(num) for num in numbers]

print(numbers)

[-9.48, -7.36, -5.23, -3.11, -1.02, 1.98, 3.94, 5.91, 7.87, 9.83, -0.12, 0.21, 0.46, 0.72, 0.98, 1.24, 1.49, 1.75, 2.02, -0.31, -0.07, 0.14, 0.28, 0.42, 0.56, 0.69]


In [25]:
!pip install pyswarms
import numpy as np
import pyswarms as ps
# from pso.cost_functions import sphere

In [26]:
initial_position = np.array(numbers)

def objective_function(x):
    return np.sum(x**2)

bounds = (np.array([-10]), np.array([10]))

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = ps.single.GlobalBestPSO(n_particles=len(numbers), dimensions=1, options=options, bounds=bounds, init_pos=initial_position.reshape(-1, 1))

best_cost, best_pos = optimizer.optimize(objective_function, iters=100)

print('Best position:', best_pos)
print('Best objective:', best_cost)


2024-06-14 18:09:31,412 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=369
2024-06-14 18:09:31,603 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 368.76386353877115, best pos: [-8.84213339]


Best position: [-8.84213339]
Best objective: 368.76386353877115


In [177]:
def sphere(x):
    return sum(x**2)

In [208]:
CostFunction = sphere  #
nVar = 10  # Number of Decision Variables
VarSize = (nVar,)  # Size of Decision Variables Matrix
VarMin = -10  # Lower Bound of Variables
VarMax = 10  # Upper Bound of Variables

In [209]:
MaxIt = 10  # Maximum Number of Iterations
nPop = np.size(numbers)  # Population Size (Swarm Size)
print(nPop)

26


In [210]:
w = 1  # Inertia Weight
wdamp = 0.99  # Inertia Weight Damping Ratio
c1 = 1.5  # Personal Learning Coefficient
c2 = 2.0  # Global Learning Coefficient

In [211]:
# Velocity Limits                        HOWW. ASK BHAIYA ABOUT THIS
VelMax = 0.1 * (VarMax - VarMin)
VelMin = -VelMax

In [212]:
particles = []

for _ in range(nPop):
    position = np.random.uniform(VarMin, VarMax, VarSize)
    velocity = np.zeros(VarSize)
    cost = CostFunction(position)
    best_position = position.copy()
    best_cost = cost
    particles.append({
        'Position': position,
        'Velocity': velocity,
        'Cost': cost,
        'Best': {
            'Position': best_position,
            'Cost': best_cost
        }
    })

In [213]:
GlobalBest = {'Position': None, 'Cost': np.inf}

for p in particles:
    if p['Cost'] < GlobalBest['Cost']:
        GlobalBest = {'Position': p['Best']['Position'].copy(), 'Cost': p['Best']['Cost']}

BestCost = np.zeros(MaxIt)

In [214]:
# for it in range(MaxIt):
#     w_values = []
#     for p in particles:
#         # Update Velocity
#         r1 = np.random.rand(*VarSize)
#         r2 = np.random.rand(*VarSize)
#         p['Velocity'] = (
#             w * p['Velocity']
#             + c1 * r1 * (p['Best']['Position'] - p['Position'])
#             + c2 * r2 * (GlobalBest['Position'] - p['Position'])
#         )

#         # Apply Velocity Limits
#         p['Velocity'] = np.maximum(p['Velocity'], VelMin)
#         p['Velocity'] = np.minimum(p['Velocity'], VelMax)

#         # Update Position
#         p['Position'] += p['Velocity']

#         # Velocity Mirror Effect
#         IsOutside = (p['Position'] < VarMin) | (p['Position'] > VarMax)
#         p['Velocity'][IsOutside] = -p['Velocity'][IsOutside]

#         # Apply Position Limits
#         p['Position'] = np.maximum(p['Position'], VarMin)
#         p['Position'] = np.minimum(p['Position'], VarMax)

#         # Evaluation
#         p['Cost'] = CostFunction(p['Position'])

#         # Update Personal Best
#         if p['Cost'] < p['Best']['Cost']:
#             p['Best']['Position'] = p['Position'].copy()
#             p['Best']['Cost'] = p['Cost']

#             # Update Global Best
#             if p['Best']['Cost'] < GlobalBest['Cost']:
#                 GlobalBest = {'Position': p['Best']['Position'].copy(), 'Cost': p['Best']['Cost']}

#     BestCost[it] = GlobalBest['Cost']

#     print(f"Iteration {it+1}: Best Cost = {BestCost[it]}")
#     w_values.append(w)

#     prompt = f"generate one positive number that is smaller than {w}, till 4 decimal places"
#     w_finder_template = f'''SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.

#     USER: {prompt}

#     ASSISTANT:
#     '''

#     w_response=lcpp_llm(prompt=w_finder_template, max_tokens=256, temperature=0.5, top_p=0.95,
#                   repeat_penalty=1.2, top_k=150,
#                   echo=True)
#     print(w_response)
#     text_response = w_response['choices'][0]['text']
#     value = re.findall(r'-?\d+\.\d+', text_response)
#     value = [float(num) for num in value]
#     print(value)

#     w = value

# print(w_values)

In [215]:
w_values = []
for it in range(MaxIt):
    for p in particles:
        # Update Velocity
        r1 = np.random.rand(*VarSize)
        r2 = np.random.rand(*VarSize)
        p['Velocity'] = (
            w * p['Velocity']
            + c1 * r1 * (p['Best']['Position'] - p['Position'])
            + c2 * r2 * (GlobalBest['Position'] - p['Position'])
        )

        # Apply Velocity Limits
        p['Velocity'] = np.maximum(p['Velocity'], VelMin)
        p['Velocity'] = np.minimum(p['Velocity'], VelMax)

        # Update Position
        p['Position'] += p['Velocity']

        # Velocity Mirror Effect
        IsOutside = (p['Position'] < VarMin) | (p['Position'] > VarMax)
        p['Velocity'][IsOutside] = -p['Velocity'][IsOutside]

        # Apply Position Limits
        p['Position'] = np.maximum(p['Position'], VarMin)
        p['Position'] = np.minimum(p['Position'], VarMax)

        # Evaluation
        p['Cost'] = CostFunction(p['Position'])

        # Update Personal Best
        if p['Cost'] < p['Best']['Cost']:
            p['Best']['Position'] = p['Position'].copy()
            p['Best']['Cost'] = p['Cost']

            # Update Global Best
            if p['Best']['Cost'] < GlobalBest['Cost']:
                GlobalBest = {'Position': p['Best']['Position'].copy(), 'Cost': p['Best']['Cost']}

    BestCost[it] = GlobalBest['Cost']

    print(f"Iteration {it+1}: Best Cost = {BestCost[it]}")

    prompt = f"generate one positive number that is smaller than {w}. Don't write anything else, just that number"
    w_finder_template = f'''SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.

    USER: {prompt}

    ASSISTANT:
    '''

    w_response = lcpp_llm(prompt=w_finder_template, max_tokens=256, temperature=0.5, top_p=0.95,
                          repeat_penalty=1.2, top_k=150,
                          echo=True)
    print(w_response)
    text_response = w_response['choices'][0]['text']
    value = re.findall(r'-?\d+\.\d+', text_response)
    value = [float(num) for num in value]
    if np.size(value) > 1:
      value = value[np.size(value) - 1]
    print(value)

    w = value
    w_values.append(w)
print(w_values)

Iteration 1: Best Cost = 115.52110733441145


Llama.generate: prefix-match hit


{'id': 'cmpl-926c25d0-9bcf-4d42-8223-b4863d7b146e', 'object': 'text_completion', 'created': 1718392356, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.\n\n    USER: generate one positive number that is smaller than 1. Don't write anything else, just that number\n\n    ASSISTANT:\n    0.9", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 60, 'completion_tokens': 3, 'total_tokens': 63}}
[0.9]
Iteration 2: Best Cost = 48.29431015880317


Llama.generate: prefix-match hit


{'id': 'cmpl-5bb5636c-03df-41f7-94ae-0e734329ce45', 'object': 'text_completion', 'created': 1718392365, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.\n\n    USER: generate one positive number that is smaller than [0.9]. Don't write anything else, just that number\n\n    ASSISTANT:\n    0.85", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 62, 'completion_tokens': 4, 'total_tokens': 66}}
0.85
Iteration 3: Best Cost = 17.282382389183446


...

{'id': 'cmpl-13212330-c970-421f-889d-e30244ae39fe', 'object': 'text_completion', 'created': 1718392425, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.\n\n    USER: generate one positive number that is smaller than 0.29. Don't write anything else, just that number\n\n    ASSISTANT:\n    0.17", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 63, 'completion_tokens': 4, 'total_tokens': 67}}
0.17
Iteration 9: Best Cost = 0.9206683059300669


Llama.generate: prefix-match hit


{'id': 'cmpl-334334af-d6af-4758-9a09-07d509011156', 'object': 'text_completion', 'created': 1718392434, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.\n\n    USER: generate one positive number that is smaller than 0.17. Don't write anything else, just that number\n\n    ASSISTANT:\n    0.169", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 63, 'completion_tokens': 5, 'total_tokens': 68}}
0.169
Iteration 10: Best Cost = 0.5431346889058778


Llama.generate: prefix-match hit


{'id': 'cmpl-4eaf555c-b97b-445d-b140-91a9570b5995', 'object': 'text_completion', 'created': 1718392441, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful, and honest assistant. Always answer as helpfully.\n\n    USER: generate one positive number that is smaller than 0.169. Don't write anything else, just that number\n\n    ASSISTANT:\n    0.123", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 64, 'completion_tokens': 5, 'total_tokens': 69}}
0.123
[[0.9], 0.85, 0.78, 0.56, 0.49, 0.37, 0.29, 0.17, 0.169, 0.123]


In [216]:
print(w_values)

[[0.9], 0.85, 0.78, 0.56, 0.49, 0.37, 0.29, 0.17, 0.169, 0.123]


In [ ]:
BestSol = GlobalBest

# Results
plt.figure()
plt.semilogy(BestCost, 'LineWidth', 2)
plt.xlabel('Iteration')
plt.ylabel('Best Cost')
plt.grid(True)
plt.show()